# Superposition

This is where I will be attemping to do superposition of the kernel onto the terma stuff.

In [1]:
import numpy as np
import sliding as sl

In [2]:
import siddon as sd
from topas2numpy import BinnedResult
from scipy import interpolate

In [3]:
'''
Parameters
'''
# for a CT array of (Nx-1,Ny-1,Nz-1) voxels
Nx = 5 
Ny = 4
Nz = 3

# distances between the x,y,z planes (also the lengths of the sides of the voxels)
dx = 0.1
dy = 2
dz = 0.5

# initial and final coordinates of the beam
x1,x2 = (0.5,0)
y1,y2 = (3,0)
z1,z2 = (2,0.25)

# initial plane coordinates
xplane1 = 0
yplane1 = 0
zplane1 = 0

# beam info and filename
ini_energy = 0.120 # in MeV
filename = 'energy_absorption_coeff.txt'

# information about the voxels
voxel_info = sd.TERMA((Nx,Ny,Nz),(dx,dy,dz),((x1,x2),(y1,y2),(z1,z2)),(xplane1,yplane1,zplane1),ini_energy,filename)

# kernel file
kernelname = '../Topas/RealKernel1.csv'

In [4]:
# real array
kernel_array = BinnedResult(kernelname).data['Sum']

In [6]:
# pretend array for testing
kernel_array = np.array([[[[0],[0],[0],[0],[0]],[[0],[0],[0],[0],[0]],[[0],[0],[0.1],[0],[0]],[[0],[0],[0],[0],[0]],[[0],[0],[0],[0],[0]]],[[[0],[0],[0],[0],[0]],[[0],[0],[0.1],[0],[0]],[[0],[0.1],[0.3],[0.1],[0]],[[0],[0],[0.1],[0],[0]],[[0],[0],[0],[0],[0]]],[[[0],[0],[0.2],[0],[0]],[[0],[0.2],[0.5],[0.2],[0]],[[0],[0.3],[1],[0.3],[0]],[[0],[0.1],[0.2],[0.1],[0]],[[0],[0],[0.1],[0],[0]]],[[[0],[0],[0],[0],[0]],[[0],[0],[0.1],[0],[0]],[[0],[0.1],[0.3],[0.1],[0]],[[0],[0],[0.1],[0],[0]],[[0],[0],[0],[0],[0]]],[[[0],[0],[0],[0],[0]],[[0],[0],[0],[0],[0]],[[0],[0],[0.1],[0],[0]],[[0],[0],[0],[0],[0]],[[0],[0],[0],[0],[0]]]])


### Interpolating Kernel

In [9]:
# still need to normalize kernel
x = np.linspace(0,len(kernel_array)-1,len(kernel_array))
y = np.linspace(0,len(kernel_array[0])-1,len(kernel_array[0]))
z = np.linspace(0,len(kernel_array[0][0])-1,len(kernel_array[0][0]))

kernel_func = interpolate.RegularGridInterpolator((x,y,z),kernel_array,bounds_error=False,fill_value=0)

### Finding Center of Kernel
*NOTE: Kernels will be forced to interact in the centre and will be an odd number of voxels.*

In [10]:
center_coor = (int(np.floor(len(kernel_array)/2)),int(np.floor(len(kernel_array[0])/2)),int(np.floor(len(kernel_array[0][0])/2)))


### Information Needed for Kernel

In [11]:
'''
Parameters
'''
# size of kernel in cm
xsize = 2
ysize = 2
zsize = 2

'''
Creating Info Dictionary
'''
kernel_info = {}
kernel_info['x'] = {}
kernel_info['y'] = {}
kernel_info['z'] = {}

kernel_info['x']['bins'] = len(kernel_array)
kernel_info['y']['bins'] = len(kernel_array[0])
kernel_info['z']['bins'] = len(kernel_array[0][0])

kernel_info['x']['size'] = xsize
kernel_info['y']['size'] = ysize
kernel_info['z']['size'] = zsize

kernel_info['x']['voxel_size'] = kernel_info['x']['size']/kernel_info['x']['bins']
kernel_info['y']['voxel_size'] = kernel_info['y']['size']/kernel_info['y']['bins']
kernel_info['z']['voxel_size'] = kernel_info['z']['size']/kernel_info['z']['bins']

In [12]:
kernel_info

{'x': {'bins': 5, 'size': 2, 'voxel_size': 0.4},
 'y': {'bins': 5, 'size': 2, 'voxel_size': 0.4},
 'z': {'bins': 5, 'size': 2, 'voxel_size': 0.4}}

### Put it All Together

In [17]:
# print(voxel)
energy_deposit = []

x_voxels = np.linspace(0,Nx-2,Nx-1)
y_voxels = np.linspace(0,Ny-2,Ny-1)
z_voxels = np.linspace(0,Nz-2,Nz-1)

voxel_array = []
for x in x_voxels:
    for y in y_voxels:
        for z in z_voxels:
            voxel_array.append(np.array([x,y,z]))
voxel_array = np.array(voxel_array)

# start of the loop
for voxel_ind in range(len(voxel_info)):
    voxel_diff = ['','','']
    for n in range(len(voxel_array)):
        voxel_diff[0] = voxel_array[n][0] - voxel_info[voxel_ind]['indices'][0]-1
        voxel_diff[1] = voxel_array[n][1] - voxel_info[voxel_ind]['indices'][1]-1
        voxel_diff[2] = voxel_array[n][2] - voxel_info[voxel_ind]['indices'][2]-1
        
        if voxel_ind == 0:
            energy_deposit.append({})
            energy_deposit[n]['indices'] = (voxel_array[n][0]+1,voxel_array[n][1]+1,voxel_array[n][2]+1)
        
        kernel_value = kernel_func((center_coor[0]+voxel_diff[0]*dx/kernel_info['x']['voxel_size'],center_coor[1]+voxel_diff[1]*dy/kernel_info['y']['voxel_size'],center_coor[2]+voxel_diff[1]*dz/kernel_info['z']['voxel_size']))
        # still need to figure out TERMA properly
        # energy_deposit[n]['energy'] += kernel_value * voxel_info[voxel_ind]['TERMA']
        
        # print(energy_deposit[n]['indices'],kernel_value)

    